In [2]:
# Verified code section
from Functions import NeuralNetwork, mse, der_mse, ReLU, der_ReLU, linear, der_linear, dataset, LeakyReLu, der_LeakyReLu, sigmoid, der_sigmoid
import numpy as np
# 1. Create a dataset
n_samples = 100
x, y_true, y_noise = dataset(n_samples)
X = x.reshape(-1, 1) # Feature matrix (n_samples, 1)

# 2. Define Network Architecture
n_inputs = 1
layer_sizes = [20, 10, 1] # Two hidden layers, one output node
activations = [ReLU, ReLU, linear]
activation_ders = [der_ReLU, der_ReLU, der_linear]
cost_func = mse
cost_der_func = der_mse

# 3. Initialize the Neural Network
for activation, activation_der in [(ReLU, der_ReLU), (LeakyReLu, der_LeakyReLu), (sigmoid, der_sigmoid)]:
    nn = NeuralNetwork(
        network_input_size=n_inputs,
        layer_output_sizes=layer_sizes,
        activation_funcs=[activation]*2 + [linear],
        activation_ders=[activation_der]*2 + [der_linear],
        cost_fun=cost_func,
        cost_der=cost_der_func,
        seed=42
    )
    # 4. Test the manual gradient
    manual_grads = nn.compute_gradient(X, y_noise)
    auto_grads = nn.autograd_gradient(X, y_noise)
    print("\n--- Comparing Gradients (L2 norm of difference) ---")
    for i in range(len(manual_grads)):
        manual_W, manual_b = manual_grads[i]
        auto_W, auto_b = auto_grads[i]

        diff_W = np.linalg.norm(manual_W - auto_W)
        diff_b = np.linalg.norm(manual_b - auto_b)

        print(f"Layer {i}:")
        print(f"  W_diff: {diff_W}")
        print(f"  b_diff: {diff_b}")

        if diff_W > 1e-10 or diff_b > 1e-10:
            print(f"  WARNING: Large difference found in layer {i}!")
        else:
            print(f"  Gradients match!")
# 5. Testing Training loop
print("\n--- Simple Training Example ---")
learning_rate = 0.001
epochs = 100

initial_cost = nn.cost(X, y_noise)
print(f"Epoch 0, Cost: {initial_cost}")

for i in range(epochs):
    # Use autograd gradient (or manual_grads, they should be the same)
    grads = nn.compute_gradient(X, y_noise)
    
    # Update weights
    grads = [(dW * learning_rate, db * learning_rate) for dW, db in grads]
    nn.update_weights(grads)
    
    if (i+1) % 20 == 0:
        cost_val = nn.cost(X, y_noise)
        print(f"Epoch {i+1}, Cost: {cost_val}")
        
final_cost = nn.cost(X, y_noise)
print(f"Training complete. Final Cost: {final_cost}")
print(f"Cost reduced: {initial_cost > final_cost}")


--- Comparing Gradients (L2 norm of difference) ---
Layer 0:
  W_diff: 1.2119809160162261e-17
  b_diff: 9.670986140218852e-17
  Gradients match!
Layer 1:
  W_diff: 2.3351598139465184e-17
  b_diff: 1.0203949608775757e-16
  Gradients match!
Layer 2:
  W_diff: 7.217888169571747e-18
  b_diff: 0.0
  Gradients match!

--- Comparing Gradients (L2 norm of difference) ---
Layer 0:
  W_diff: 1.3444106938820255e-17
  b_diff: 8.708242736640235e-17
  Gradients match!
Layer 1:
  W_diff: 2.7305707463421613e-17
  b_diff: 9.464432931723923e-17
  Gradients match!
Layer 2:
  W_diff: 1.033609230195259e-17
  b_diff: 1.1102230246251565e-16
  Gradients match!

--- Comparing Gradients (L2 norm of difference) ---
Layer 0:
  W_diff: 3.436319130918789e-18
  b_diff: 1.6719227153954813e-17
  Gradients match!
Layer 1:
  W_diff: 2.0852790466893888e-16
  b_diff: 1.0460253102363242e-16
  Gradients match!
Layer 2:
  W_diff: 3.477763656540197e-16
  b_diff: 0.0
  Gradients match!

--- Simple Training Example ---
Epoch 0

### Testing Regression network against Scikit-learn MLPRegressor

In [ ]:
# Verified code section
from Functions import NeuralNetwork, mse, der_mse, ReLU, der_ReLU, linear,sigmoid, der_sigmoid, der_linear, dataset
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
# 1. Create a dataset
n_samples = 100
x, y_true, y_noise = dataset(n_samples)
X = x.reshape(-1, 1) # Feature matrix (n_samples, 1)

scaler = StandardScaler()
X = scaler.fit_transform(X)
y = y_true - np.mean(y_true)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# 2. Define Network Architecture
n_inputs = 1
layer_sizes = [20, 10, 1] # Two hidden layers, one output node
activations = [sigmoid, sigmoid, linear]
activation_ders = [der_sigmoid, der_sigmoid, der_linear]
cost_func = mse
cost_der_func = der_mse

# 3. Initialize the Neural Network
nn = NeuralNetwork(
    network_input_size=n_inputs,
    layer_output_sizes=layer_sizes,
    activation_funcs=activations,
    activation_ders=activation_ders,
    cost_fun=cost_func,
    cost_der=cost_der_func,
    seed=42
)
# 5. Testing Training loop
print("--- Simple Training Comparison with scikit-learn ---")
learning_rate = 0.1
epochs = 1000



for i in range(epochs):
    # Use autograd gradient (or manual_grads, they should be the same)
    grads = nn.compute_gradient(X_train, y_train)
    
    # Update weights
    grads = [(dW * learning_rate, db * learning_rate) for dW, db in grads]
    nn.update_weights(grads)


        

final_cost = nn.cost(X_test, y_test)

from sklearn.neural_network import MLPRegressor
# store models for later use
dnn = MLPRegressor(hidden_layer_sizes=(20,10), activation='logistic', solver='sgd',
                             learning_rate_init=learning_rate, learning_rate='constant', max_iter=epochs, random_state=42,
                             batch_size=X_train.shape[0],
                             alpha=0.000001,
                             tol=1e-10000,
                             early_stopping=False, verbose=False)
dnn.fit(X_train, y_train)

# Get Test MSE from your nn

print(f"Your NeuralNetwork Test MSE: {final_cost:.6f}")

# Get Test MSE from scikit-learn's dnn
dnn_pred = dnn.predict(X_test)
dnn_test_mse = mse(y_test, dnn_pred)
print(f"SKLearn MLPRegressor Test MSE: {dnn_test_mse:.6f}")

--- Simple Training Comparison with scikit-learn ---


c:\Users\Anton\Documents\UiO Fag\FYS-STK4155 Anvendt dataanalyse og maskinlæring\Eget Arbeid\.venv\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1771: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Your NeuralNetwork Test MSE: 0.071445
SKLearn MLPRegressor Test MSE: 0.071389


c:\Users\Anton\Documents\UiO Fag\FYS-STK4155 Anvendt dataanalyse og maskinlæring\Eget Arbeid\.venv\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:781: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
